## Sum WorldPop Population by Sargassum Analysis Segment

In [ ]:
# Import GEE & initialize
import os
import ee
import geemap
from pprint import pprint
# Initialize GEE
try:
    ee.Initialize()
except Exception as e:
    ee.Authenticate()
    ee.Initialize()
import numpy as np
import pandas as pd

## Add / Import Data sets

### Shoreline Segments

In [ ]:
# Shoreline Segments
shore_segments = ee.FeatureCollection('users/allison_bailey/sargassum/shoreQR_segments_20210331')
print(shore_segments.size().getInfo())

### WorldPop 2015-2020

In [ ]:
# WorldPop -- Mexico, 2015-2020
worldpop_coll = ee.ImageCollection('WorldPop/GP/100m/pop')\
        .filter(ee.Filter.inList('country',ee.List(['MEX'])))\
        .filterDate('2015-01-01', '2020-12-31') 
# print(worldpop.first().propertyNames().getInfo())
# print(worldpop.first().getInfo())

In [ ]:
worldpop_stack = worldpop_coll.toBands()
# pprint(worldpop_stack.getInfo())

### Agreggating tips https://spatialthoughts.com/2021/05/13/aggregating-population-data-gee/

In [ ]:
# At 100 meters resolution, there are a lot of areas where the segments do not intersect the population data
# So, Aggregate Population raster to 500m 

# Get the projection at required scale
wpproj = ee.Image(worldpop_coll.first()).projection()
projectionAt500m = ee.Image(worldpop_coll.first()).projection().atScale(500)
print(wpproj.getInfo())
print(projectionAt500m.crs().getInfo())

worldpop_500m = worldpop_stack.reduceResolution(**{
    'reducer': ee.Reducer.sum().unweighted(),
    'maxPixels': 1024
  }).reproject(**{
    'crs': projectionAt500m,
  })
print(worldpop_500m.projection().getInfo())

## Summarize Population by Segment

In [ ]:
# Sum the annual worldpop pixels for each segment -- 100 m (source data native resolution)
segment_wpsum_100m = worldpop_stack.reduceRegions(**{
  'collection': shore_segments,
  'reducer': ee.Reducer.sum(),
  'scale': 100,
})

segment_wpsum_500m = worldpop_500m.reduceRegions(**{
  'collection': shore_segments,
  'reducer': ee.Reducer.sum(),
  'scale': 500,
})
# Print the first feature, to illustrate the result.
# pprint(segment_wpsum_100m.first().getInfo())
# pprint(segment_wpsum_500m.first().getInfo())

## Visualization

In [ ]:
# Map = geemap.Map(center=[20,-87], zoom=8)  # Center on Quintana Roo
Map = geemap.Map(center=[21.1,-86.8], zoom=12)  #   Zoomed in 

# Basemap
# Map.add_basemap('Esri Ocean')

#WorldPop Raster
visualization = {
  'bands': ['population'],
  'min': 0.0,
  'max': 50.0,
  'palette': ['24126c', '1fff4f', 'd4ff50']
};
Map.addLayer(worldpop_coll, visualization, 'Population Collection');
Map.addLayer(worldpop_stack, {}, 'Population Image',False);
Map.addLayer(worldpop_500m, {}, '500m Population Image',False);


# Shoreline Segment vectors
Map.addLayer(segment_wpsum_100m, {}, 'Shoreline Segments', False)

# Display outline of Nearshore Mask
# Create an empty image into which to paint the features, cast to byte.
empty = ee.Image().byte();
# Paint all the polygon edges with the same number and width, display.
segments_image = empty.paint(**{
  'featureCollection': segment_wpsum_100m,
  'color': 1,
  'width': 1.5
});
Map.addLayer(segments_image,  {'palette':'orange'}, 'Shoreline Segments Img', True); # orange

Map

## Exporting

In [ ]:
# Pull out essential properties to create final table
# This function creates a new feature with null geometry, since we don't need location
def outTable(feature):
    outfeature = ee.Feature(None, { 
    'seg_id' : feature.get('seg_id'),
    'pop_sum2015': feature.get('MEX_2015_population'),
    'pop_sum2016': feature.get('MEX_2016_population'),
    'pop_sum2017': feature.get('MEX_2017_population'),
    'pop_sum2018': feature.get('MEX_2018_population'),
    'pop_sum2019': feature.get('MEX_2019_population'),
    'pop_sum2020': feature.get('MEX_2020_population'),
    })
    return outfeature

# Create Final table with key properties for export
segment_wpsum_500m_output = segment_wpsum_500m.map(outTable)
print(segment_wpsum_500m_output.first().getInfo())

In [ ]:
#  Convert GEE ouptut feature class (with null geometry) to Pandas data frame
pop_columns = ['seg_id','pop_sum2015', 'pop_sum2016', 'pop_sum2017', 'pop_sum2018', 'pop_sum2019', 'pop_sum2020']
def fc_to_df(fc, outcols):
    listlen = len(outcols)
    nested_list = fc.reduceColumns(ee.Reducer.toList(len(outcols)), outcols).values().get(0)
    df = pd.DataFrame(nested_list.getInfo(), columns=outcols)
    return df
wpsum500m_df = fc_to_df(segment_wpsum_500m_output, pop_columns)
wpsum500m_df

In [ ]:
out_dir = '/Users/arbailey/natcap/idb/data/work/sargassum/shore_segments'
wpsum500m_df.to_csv(os.path.join(out_dir,'shoreQR_segments_pop500m.csv'),header=pop_columns, index=False)